In [1]:
import pandas as pd
import langid
import requests
import re
import pickle

In [2]:
with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/books_data.pkl', 'rb') as f:
    books_data = pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/ratings_data.pkl', 'rb') as f:
    ratings_data= pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/users_data.pkl', 'rb') as f:
    users_data= pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/books_ratings.pkl', 'rb') as f:
    books_ratings= pickle.load(f)

In [3]:
def book_rating(ISBN):
   # Check if authors_books is not empty before accessing the first row
    if not books_data.loc[books_data['ISBN'] == ISBN, ['Book-Title', 'Book-Author', 'Language']].empty:
        # Get the title of the book, author, and language with the given ISBN
        book_title, book_author, book_language = books_data.loc[books_data['ISBN'] == ISBN, ['Book-Title', 'Book-Author', 'Language']].iloc[0]

        # Get all the books that the author wrote
        authors_books = books_data.loc[(books_data['Book-Author'] == book_author) & (books_data['Language'] == book_language), ['ISBN', 'Book-Title', 'Language']]

        # Filter the DataFrame for books with titles whose leftmost 30 characters are the same as translated_title
        matched_books = authors_books.loc[authors_books['Book-Title'].str[:30] == book_title[:30]]

        books_rating = ratings_data.loc[ratings_data['ISBN'].isin(matched_books['ISBN']), 'Book-Rating'].mean()

        return books_rating
    else:
        return None

In [4]:
def recommendation(BookIsLiked, BookTitle, BookAuthor):
    ## find the lord of the Rings books
    filtered_books = books_data.loc[books_data["Book-Title"].str.contains(BookTitle) & books_data["Book-Author"].str.contains(BookAuthor)]

    ## create merged dataframes with the Lord of the Rings books
    users_ratings_books = pd.merge(left=ratings_data, right= filtered_books, left_on = "ISBN", right_on = "ISBN")
    ##users_ratings_books

    ## getting the users that like Lord of the Rings
    users_ratings_books_filtered = users_ratings_books.loc[users_ratings_books["Book-Rating"] > BookIsLiked]
    lord_readers = pd.unique(users_ratings_books_filtered['User-ID'])
    ## print(lord_readers)
   
    ## getting the ISBN and Book-Ratings of the books that Lord of the Rings lovers also read and are in the same language
    books = ratings_data[ratings_data['User-ID'].isin(lord_readers)].reindex(columns = ['ISBN', 'Book-Rating'])
    books['Language'] = books.merge(books_data, on='ISBN', how='left')['Language']
    books = books.loc[books['Language'].isin(filtered_books['Language'])]
    books['Book-Rating'] = books['ISBN'].apply(book_rating)

    ## keeping only the books where there is more than 10 ratings
    count = books.groupby('ISBN')['Book-Rating'].count().reset_index(name = 'Count')
    count = count.loc[count["Count"] > 10]
    ## count.sort_values(by='Count', ascending=False)
    ## grouped_data['Count'].sum()

    ## keeping only the books where the mean of the ratings is bigger than BookIsLiked, but the book should not be Lord of the Rings book
    ratings = pd.merge(left=ratings_data,right=count, left_on="ISBN", right_on="ISBN")
    ratings = ratings.groupby('ISBN')['Book-Rating'].mean().reset_index(name = 'MeanRating').sort_values(by='MeanRating', ascending=False)
    ratings = ratings.loc[ratings["MeanRating"] > BookIsLiked]
    ratings = ratings[ratings.ISBN.isin(filtered_books["ISBN"])==False]
    ##ratings

    ## get the selected book data
    books_data[books_data['ISBN'].isin(ratings['ISBN'])]
    print(books_data[books_data['ISBN'].isin(ratings['ISBN'])])

In [5]:
## defining variables
BookIsLiked = 6 ## what does it mean that I like the book? How high the rating must be?
BookTitle = 'The Fellowship of the Ring'.upper()
BookAuthor = 'TOLKIEN'

In [ ]:
%%time
recommendation(BookIsLiked, BookTitle, BookAuthor)